<a href="https://colab.research.google.com/github/MaggieAppleton/Colab-Notebooks/blob/main/Orchestrator_Workers_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orchestrator-Workers Workflow Learning Exercises

In the orchestrator-workers pattern, a central LLM breaks down complex tasks and delegates subtasks to specialized worker LLMs.

**Key Concepts:**
- **Orchestrator**: Analyzes tasks and creates subtask breakdowns
- **Workers**: Specialized LLMs that handle specific aspects
- **Synthesis**: Combining worker results into final output

**When to use:** Complex tasks where subtasks can't be predicted in advance, like multi-file coding changes or comprehensive research.

## Setup

In [ ]:
!pip install anthropic

In [ ]:
import anthropic
import re
import json
from google.colab import userdata

# Initialize the Anthropic client
claude_api_key = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=claude_api_key)
model = "claude-3-5-haiku-latest"

def call_claude(prompt):
    response = client.messages.create(
        model=model,
        max_tokens=800,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

def extract_xml(text, tag, list=False):
    """Extract content from XML tags
        Returns:
        String (if list=False) or List of strings (if list=True)
        Empty string or empty list if no matches found
    """
    pattern = f'<{tag}>(.*?)</{tag}>'
    if list:
        matches = re.findall(pattern, text, re.DOTALL)
        return [match.strip() for match in matches]
    else:
        match = re.search(pattern, text, re.DOTALL)
        return match.group(1).strip() if match else ""


print("Setup complete!")

Setup complete!


# ✅ Exercise 1: Basic Task Breakdown (Warm-up)

Start with the core skill: getting an orchestrator to break down a task.

**Your Goal:** Create a function that takes a complex task and breaks it into 2-4 subtasks.

**Hints:**
- Ask the orchestrator to analyze the task first
- Request subtasks in a structured format (XML works well)
- Each subtask should have a type/category and description

In [ ]:
def orchestrator_breakdown(task):
    """Break down a complex task into manageable subtasks"""

    orchestrator_prompt = f"""
    Analyse this task and think carefully about how you would break it down into 2-4 distinct subtasks that can be completed independently.

    <analysis>
    Write an analysis of what this task involves and what your should take into consideration while breaking it down into subtasks.
    </analysis>

    <subtask>
      <name>Give each subtask a name</name>
      <type>
      Give each subtask a type (research, budget, logistics, etc.)
      </type>
      <description>
      Describe the subtask
      </description>
    </subtask>

    Task: {task}
    """

    response = call_claude(orchestrator_prompt)

    # Extract the analysis and subtasks
    analysis = extract_xml(response, 'analysis')
    subtasks_list = extract_xml(response, 'subtask', list)

    subtasks = parse_subtasks(subtasks_list)

    print(('-' * 20) + ' ANALYSIS ' + ('-' * 20))
    print(f"{analysis}\n\n")

    print(('-' * 20) + ' SUBTASKS ' + ('-' * 20))
    for i, subtask in enumerate(subtasks, 1):
      print(f"--- Task {i} ---")
      print(f"Name: {subtask['name']}")
      print(f"Type: {subtask['type']}")
      print(f"Description: {subtask['description']}\n")

    return analysis, subtasks

def parse_subtasks(subtasks):
    """Parse XML subtasks into a list of dictionaries"""

    # TODO: Parse XML like:
    # <subtask><type>research</type><description>Find info about X</description></subtask>
    # Return list of {"name": thingy, "type": "research", "description": "Find info about X"}
    subtasks_list = []
    for subtask in subtasks:
      results = {}
      name = extract_xml(subtask, 'name')
      type = extract_xml(subtask, 'type')
      description = extract_xml(subtask, 'description')

      subtasks_list.append(
          {
              'name': name,
              'type': type,
              'description': description
          }
      )

    # print(f"Results: {subtasks_list}")

    return subtasks_list

# Test it out
task = "Plan a 5-day trip to Japan for first-time visitors interested in culture and food"
analysis, subtasks = orchestrator_breakdown(task)

-------------------- ANALYSIS --------------------
To effectively break down this task, we need to consider several key factors:
1. The specific interests of first-time visitors (culture and food)
2. Logistical constraints of a 5-day trip
3. Geographic considerations within Japan
4. Seasonal variations and travel practicalities
5. Budget and travel style considerations

The goal is to create subtasks that can be developed independently but ultimately integrate into a cohesive trip plan. Each subtask should address a critical aspect of trip planning while allowing for flexibility and personal customization.


-------------------- SUBTASKS --------------------
--- Task 1 ---
Name: Destination and Itinerary Research
Type: Research
Description: Identify key cultural and culinary destinations in Japan, focusing on regions that can be reasonably explored within 5 days. This includes researching cities like Tokyo, Kyoto, and surrounding areas that offer rich cultural experiences and renowned 

# ✅ Exercise 2: Worker Execution (Building Up)

Now implement workers that can handle the subtasks from your orchestrator.

**Your Goal:** Create specialized workers that execute subtasks and return detailed results.

**Hints:**
- Workers should be told their specialty/role
- Give them the original task for context
- Make them focus on their specific subtask

In [ ]:
def worker_execute(original_task, subtask_type, subtask_name, subtask_description):
    """Execute a specific subtask with a specialized worker"""

    worker_prompt = f"""
    You are a specialist in {subtask_type}.
    We need your help executing one small part of this larger task: {original_task}.
    Please focus on this specific sub-task: {subtask_name}.
    Sub-task description: {subtask_description}

    Return detailed, actionable outputs to help us make progress on this task and return it within <result> tags.
    """

    # Make the LLM call and extract result
    response = call_claude(worker_prompt)
    result = extract_xml(response, 'result')

    return result

def simple_orchestrator_workflow(task):
    """Complete workflow: break down task and execute with workers"""

    # Step 1: Break down the task
    analysis, subtasks = orchestrator_breakdown(task)

    # Step 2: Execute each subtask with workers
    results = []

    # Loop through subtasks and execute each with a worker
    # Collect results in a list
    for subtask in subtasks:
      print(f"\n---- WORKER: {subtask['name']} ----")

      subtask_result = worker_execute(task, subtask['type'], subtask['name'], subtask['description'])
      results.append({
          'name': subtask['name'],
          'description': subtask['description'],
          'result': subtask_result
          })

      print(f"\nRESULT: {subtask_result}")

    return {
        "analysis": analysis,
        "subtasks": subtasks,
        "results": results
    }

# Test the complete workflow
task = "Plan a weekend team retreat focused on problem-solving and collaboration"
result = simple_orchestrator_workflow(task)
print(f"\n=== WORKFLOW COMPLETE ===")
print(f"Completed {len(result['results'])} subtasks")

-------------------- ANALYSIS --------------------
Analyzing this task requires considering multiple dimensions:
- Purpose of the retreat (problem-solving and collaboration)
- Logistical requirements (location, duration, timing)
- Team dynamics and participant needs
- Budget constraints
- Specific activities that will achieve the retreat's goals
- Practical planning elements like transportation, accommodation, and scheduling

Key considerations:
1. The retreat needs a clear strategic objective beyond just socializing
2. Activities must be designed to genuinely improve team problem-solving skills
3. Venue and format should encourage open communication
4. Balance structured activities with flexible interaction time
5. Ensure inclusivity and engagement for all team members


-------------------- SUBTASKS --------------------
--- Task 1 ---
Name: Retreat Location and Logistics Planning
Type: Logistics
Description: Research and select an appropriate venue that supports collaborative activit

# Exercise 3: Dynamic Task Adaptation (Intermediate)

Make your orchestrator smarter by having it adapt its breakdown based on task characteristics.

**Your Goal:** Create an orchestrator that analyzes different types of tasks and creates appropriate subtask breakdowns.

**Challenge:** Test with very different tasks (research, planning, creative work) and see how the breakdown adapts.

**Hints:**
- Ask the orchestrator to first identify what TYPE of task this is
- Let it decide how many subtasks are needed (2-5 range)
- Different task types should get different subtask categories

In [ ]:
def adaptive_orchestrator(task):
    """Intelligently break down tasks based on their nature"""

    # TODO: Create a prompt that:
    # 1. Asks the LLM to first analyze what TYPE of task this is
    # 2. Based on that, decide appropriate subtasks (2-5 subtasks)
    # 3. Explain the reasoning

    adaptive_prompt = """
    # Your adaptive prompt here
    # Include: <task_type>, <reasoning>, <subtasks>
    """

    # TODO: Extract task_type, reasoning, and subtasks
    response = None
    task_type = ""
    reasoning = ""
    subtasks = []

    print(f"\n=== TASK ANALYSIS ===")
    print(f"Type: {task_type}")
    print(f"Reasoning: {reasoning}")

    return task_type, reasoning, subtasks

def adaptive_worker(original_task, subtask_type, subtask_description, task_context):
    """Worker that adapts its approach based on the task context"""

    # TODO: Create a worker that's aware of the overall task type
    # This should help it provide more appropriate responses

    adaptive_worker_prompt = """
    # Your adaptive worker prompt here
    # Include task context in the prompt
    """

    response = None
    result = ""

    return result

def adaptive_workflow(task):
    """Complete adaptive workflow"""

    # Get adaptive breakdown
    task_type, reasoning, subtasks = adaptive_orchestrator(task)

    # Execute with context-aware workers
    results = []

    # TODO: Execute subtasks with the adaptive worker
    # Pass the task_type as context

    return {
        "task_type": task_type,
        "reasoning": reasoning,
        "subtasks": subtasks,
        "results": results
    }

# Test with different types of tasks
test_tasks = [
    "Research the pros and cons of remote work for software teams",
    "Write a short story about a robot learning to paint",
    "Create a marketing strategy for a new productivity app",
    "Debug why our Python web scraper is running slowly"
]

for task in test_tasks:
    print(f"\n{'='*60}")
    print(f"TASK: {task}")
    result = adaptive_workflow(task)
    print(f"Identified as: {result['task_type']}")

# Exercise 4: Research with Synthesis (Advanced)

Build a complete research workflow that not only breaks down and executes research tasks, but synthesizes findings into a coherent final report.

**Your Goal:** Create a workflow that researches a topic from multiple angles and combines findings intelligently.

**Key Challenge:** The synthesis step - combining multiple worker outputs into something more valuable than the sum of parts.

**Hints:**
- Research tasks often need: background, comparisons, pros/cons, current trends
- Synthesis should look for patterns, conflicts, and conclusions across worker outputs
- Consider asking for specific output formats (bullet points, tables, recommendations)

In [ ]:
def research_orchestrator(research_question):
    """Break down research questions into investigative subtasks"""

    # TODO: Create a research-focused orchestrator that:
    # 1. Identifies what kind of research this is
    # 2. Creates 2-4 research subtasks that cover different angles
    # 3. Explains how these subtasks will build toward answering the question

    research_prompt = """
    # Your research orchestrator prompt here
    # Should output: <research_type>, <strategy>, <subtasks>
    """

    response = None
    research_type = ""
    strategy = ""
    subtasks = []

    print(f"\n=== RESEARCH STRATEGY ===")
    print(f"Type: {research_type}")
    print(f"Strategy: {strategy}")

    return research_type, strategy, subtasks

def research_worker(research_question, subtask_type, subtask_description):
    """Specialized research worker"""

    # TODO: Create a research worker that:
    # 1. Understands it's doing research (not just any task)
    # 2. Focuses on finding factual, current information
    # 3. Structures findings clearly
    # 4. Notes any limitations or uncertainties

    research_worker_prompt = """
    # Your research worker prompt here
    """

    response = None
    findings = ""

    return findings

def synthesize_research(research_question, research_results):
    """Combine research findings into a coherent analysis"""

    # TODO: Create a synthesis prompt that:
    # 1. Takes all the research worker outputs
    # 2. Looks for patterns, agreements, and conflicts
    # 3. Draws conclusions that answer the original question
    # 4. Identifies gaps or limitations
    # 5. Provides actionable insights

    # Format all results for the synthesis prompt
    formatted_results = ""
    for result in research_results:
        formatted_results += f"\n\n=== {result['type'].upper()} FINDINGS ===\n{result['result']}"

    synthesis_prompt = f"""
    # Your synthesis prompt here
    # Include the research_question and formatted_results
    """

    response = None
    synthesis = ""

    return synthesis

def complete_research_workflow(research_question):
    """End-to-end research workflow with synthesis"""

    # Step 1: Plan the research
    research_type, strategy, subtasks = research_orchestrator(research_question)

    # Step 2: Execute research subtasks
    research_results = []

    # TODO: Execute each research subtask

    # Step 3: Synthesize findings
    print(f"\n=== SYNTHESIS ===")
    final_analysis = synthesize_research(research_question, research_results)
    print(final_analysis)

    return {
        "research_type": research_type,
        "strategy": strategy,
        "subtasks": subtasks,
        "findings": research_results,
        "synthesis": final_analysis
    }

# Test with a research question
research_question = "What are the key factors to consider when choosing between different password managers for a small business?"
result = complete_research_workflow(research_question)

---

# Answer Key / Reference Implementations

Use these if you get stuck or want to compare your approach. Try implementing on your own first!

## Reference: Basic XML Parsing

In [ ]:
def reference_parse_subtasks(subtasks_xml):
    """Reference implementation for parsing XML subtasks"""
    subtasks = []
    current_task = {}

    for line in subtasks_xml.split('\n'):
        line = line.strip()
        if not line:
            continue

        if '<subtask>' in line:
            current_task = {}
        elif '<type>' in line:
            current_task["type"] = line.replace('<type>', '').replace('</type>', '').strip()
        elif '<description>' in line:
            current_task["description"] = line.replace('<description>', '').replace('</description>', '').strip()
        elif '</subtask>' in line:
            if "type" in current_task and "description" in current_task:
                subtasks.append(current_task)

    return subtasks

## Reference: Sample Orchestrator Prompt

In [ ]:
def reference_orchestrator_prompt(task):
    """Reference orchestrator prompt structure"""
    return f"""
You are a task orchestrator. Break down this complex task into 2-4 manageable subtasks.

Task: {task}

First, analyze what this task requires, then break it into logical subtasks.

<analysis>
Explain your reasoning for how you're breaking this down.
</analysis>

<subtasks>
    <subtask>
    <type>research</type>
    <description>Research destinations and activities</description>
    </subtask>
    <subtask>
    <type>logistics</type>
    <description>Plan transportation and accommodation</description>
    </subtask>
    <subtask>
    <type>budget</type>
    <description>Create cost estimates and budget breakdown</description>
    </subtask>
</subtasks>
"""

## Reference: Sample Worker Prompt

In [ ]:
def reference_worker_prompt(original_task, subtask_type, subtask_description):
    """Reference worker prompt structure"""
    return f"""
You are a specialist in {subtask_type}. You're helping with a larger project.

Original task: {original_task}
Your specific focus: {subtask_description}

Provide detailed, actionable information for your area of expertise.
Be specific and practical in your recommendations.

<result>
[Your specialized response here]
</result>
"""